In [16]:
from powersimdata import Scenario
import pyarrow.feather as feather
import pandas as pd
from postreise.analyze.transmission.utilization import (
    generate_cong_stats,
    get_utilization,
)
import numpy as np

In [3]:
def get_grid_branch(grid, pf):
    branch = grid.branch.loc[(grid.branch['rateA'] != 0) 
                             & (grid.branch['branch_device_type'] == 'Line')]

    branch = branch[['rateA', 'from_bus_id', 'to_bus_id', 'interconnect']]
    branch = branch.rename(columns={ 'rateA': 'capacity' })

    branch = bus2coords(branch, grid, bus_col_name='from_bus_id', col_prefix="from_")
    branch = bus2coords(branch, grid, bus_col_name='to_bus_id', col_prefix="to_")
    branch = combine_cols(branch, ['from_coords', 'to_coords'], 'coords')
    branch = combine_cols(branch, ['from_zone', 'to_zone'], 'zone')
    branch = branch.drop(columns=['from_interconnect', 'to_interconnect'])

    print("getting util")
    # UTILIZATION DATA
    util_median = get_utilization(grid.branch, pf, median=True).T
    util_median = util_median.rename(columns={0: 'median_utilization'})

    print("getting congestion stats")

    # RISK DATA (congestion)
    congestion_stats = generate_cong_stats(pf, grid.branch)[['risk', 'bind']]

    # Combine
    util_and_cong = pd.concat([util_median, congestion_stats], axis=1)

    # Replace any NANs with 0 for binding and risk
    util_and_cong = util_and_cong.fillna(0)

    # Set index as col so we keep the branch id when writing records to dict
    util_and_cong = util_and_cong.reset_index()
    branch_with_util_and_cong = branch.join(util_and_cong, on='branch_id')
    branch_with_util_and_cong = branch_with_util_and_cong.reset_index()
    
    print()
    print(branch_with_util_and_cong)
    
    return branch_with_util_and_cong

In [4]:
def bus2coords(
    df, 
    grid, 
    bus_col_name='bus_id', 
    col_prefix="", 
    col_suffix="", 
    drop_bus_col=True, # Should be true if we are coordinate rounding
    coordinate_rounding=0, 
    groupby_cols=[], # for coord rounding
    agg_method={}    # for coord rounding
):
    bus = grid.bus[["lon", "lat", "zone_id", "interconnect"]]
    bus = bus.rename(columns={ 'zone_id': 'zone' })
    
    # Get zone info
    bus['zone'] = bus['zone'].replace(grid.id2zone)
    bus['zone'] = bus['zone'].replace(loadzone2state)
    
    # We need to update column names before joining tables
    bus = bus.add_prefix(col_prefix).add_suffix(col_suffix)
        
    # Get new column names for later
    # TODO: is there a more elegant way to do this?
    lat_col = get_pref_suf("lat", col_prefix, col_suffix)
    lon_col = get_pref_suf("lon", col_prefix, col_suffix)
    interconnect_col = get_pref_suf("interconnect", col_prefix, col_suffix)
    zone_col = get_pref_suf("zone", col_prefix, col_suffix)
    coords_col = get_pref_suf("coords", col_prefix, col_suffix)
    
    # Add location info to original df
    new_df = df.join(bus, bus_col_name)
    if (drop_bus_col):
        new_df = new_df.drop(columns=bus_col_name)
    
    # If we are coordinate_rounding, combine rows by location and groupby_cols 
    # Aggregate other cols by agg_method
    if (coordinate_rounding):
        new_df[lat_col] = new_df[lat_col].round(coordinate_rounding)
        new_df[lon_col] = new_df[lon_col].round(coordinate_rounding)

        new_df = new_df.groupby([lat_col, lon_col, interconnect_col, zone_col] + groupby_cols).agg(agg_method)
        new_df = new_df.reset_index()
    
    # combine lat and lon into coords col
    new_df = combine_cols(new_df, [lon_col, lat_col], coords_col)
    return new_df

In [5]:
def get_pref_suf(string, prefix, suffix):
    return f"{prefix}{string}{suffix}"

In [6]:
loadzone2state = {
    "Bay Area": "California",
    "Central California": "California",
    "Northern California": "California",
    "Southeast California": "California",
    "Southwest California": "California",

    "Florida Panhandle": "Florida",
    "Florida North": "Florida",
    "Florida South": "Florida",

    "Georgia North": "Georgia",
    "Georgia South": "Georgia",

    "Chicago North Illinois": "Illinois",
    "Illinois Downstate": "Illinois",

    "Michigan Northern": "Michigan",
    "Michigan Southern": "Michigan",

    "Minnesota Northern": "Minnesota",
    "Minnesota Southern": "Minnesota",

    "Missouri East": "Missouri",
    "Missouri West": "Missouri",

    "Montana Eastern": "Montana",
    "Montana Western": "Montana",

    "New York City": "New York",
    "Upstate New York": "New York",

    "Western North Carolina": "North Carolina",

    "New Mexico Eastern": "New Mexico",
    "New Mexico Western": "New Mexico",

    "Ohio River": "Ohio",
    "Ohio Lake Erie": "Ohio",

    "Pennsylvania Eastern": "Pennsylvania",
    "Pennsylvania Western": "Pennsylvania",

    "Coast": "Texas",
    "East": "Texas",
    "East Texas": "Texas",
    "El Paso": "Texas",
    "Far West": "Texas",
    "North": "Texas",
    "North Central": "Texas",
    "South": "Texas",
    "South Central": "Texas",
    "Texas Panhandle": "Texas",
    "West": "Texas",

    "Virginia Mountains": "Virginia",
    "Virginia Tidewater": "Virginia",
}

In [7]:
# Combines columns into a new one. Drops old columns.
def combine_cols(df, cols, new_col_name):
    df_copy = df.copy()
    df_copy[new_col_name] = df_copy[cols].values.tolist()
    return df_copy.drop(columns=cols)

In [10]:
scenario = Scenario("1705")
pf_usa = scenario.state.get_pf()
grid_stats = get_grid_branch(scenario.state.get_grid(), pf_usa)
grid_stats

Failed to download ScenarioList.csv from server
Falling back to local cache...
Failed to download ExecuteList.csv from server
Falling back to local cache...
SCENARIO: Julia | USA2030_Ambitious_Design1_OB0.5_Warmstart2_Mesh500x8t100

--> State
analyze
--> Loading grid
Failed to download ScenarioList.csv from server
Falling back to local cache...
--> Loading ct
--> Loading PF
getting util
getting congestion stats
Removing non line branches
Removing lines that never are >75% utilized
Getting utilization
Counting hours
Getting fraction of hours above threshold
Calculating binding hours
Calculating risk
Combining branch and utilization info
Calculating distance and finalizing results

       branch_id  capacity interconnect  \
0              4    186.85      Eastern   
1              6    531.98      Eastern   
2              7    600.42      Eastern   
3             10    509.38      Eastern   
4             11    401.34      Eastern   
...          ...       ...          ...   
61444     

,branch_id,capacity,interconnect,coords,zone,index,median_utilization,risk,bind
0,4,186.85,Eastern,"[[-70.3062, 43.9376], [-70.2211, 43.9761]]","[Maine, Maine]",4,0.082517,0.0,0.0
1,6,531.98,Eastern,"[[-70.1775, 43.883], [-70.2211, 43.9761]]","[Maine, Maine]",6,0.224559,0.0,0.0
2,7,600.42,Eastern,"[[-70.2211, 43.9761], [-70.1876, 44.1104]]","[Maine, Maine]",7,0.233386,0.0,0.0
3,10,509.38,Eastern,"[[-70.3053, 44.101], [-70.2236, 44.1654]]","[Maine, Maine]",10,0.016501,0.0,0.0
4,11,401.34,Eastern,"[[-70.3053, 44.101], [-70.4352, 44.1625]]","[Maine, Maine]",11,0.218715,0.0,0.0
...,...,...,...,...,...,...,...,...,...
61444,104187,100.00,Western,"[[-124.324466, 45.43589], [-123.787, 45.3609]]","[Oregon, Oregon]",104187,0.000000,0.0,0.0
61445,104188,100.00,Western,"[[-124.349185, 46.576647], [-123.983, 46.4983]]","[Washington, Washington]",104188,0.000000,0.0,0.0
61446,104189,100.00,Western,"[[-124.478275, 46.979165], [-124.172, 46.9275]]","[Washington, Washington]",104189,0.000000,0.0,0.0
61447,104190,100.00,Western,"[[-124.655429, 47.534671], [-124.184, 47.6956]]","[Washington, Washington]",104190,0.000000,0.0,0.0


In [17]:
branch = scenario.state.get_grid().branch
for i in range(int(pf_usa.shape[0]/24)):
    print(i)
    pf_usa_day = pf_usa.iloc[(i*24):((i+1)*24)]
    day_util = pf_usa_day.abs().divide(branch.rateA).replace(np.inf, 0)
    pruned_util = day_util.T.loc[grid_stats["branch_id"].values]
    pruned_util = pruned_util.astype('float32')
    feather.write_feather(pruned_util, f'util{i}.arrow', compression='uncompressed')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
import gzip

In [30]:
 for i in range(int(pf_usa.shape[0]/24)):
    # read binary
    with open(f'util{i}.arrow', 'rb') as f:
        content = f.read()
    # write compressed file
    with gzip.open(f'util{i+1}.arrow.gz', 'wb') as f:
        f.write(content)

In [24]:
import gzip
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContentSettings

In [26]:
class BlobUtil(object):
    """
    Upload figures/other files to blob storage for website
    """

    def __init__(self, connection_string, container_name):
        """
        :param connection_string: str needed in order to access storage account container
            You can find the connection string in the
            azure portal > storage accounts > access keys
        :param container_name: str name of blob storage container.
            In Storage Explorer this is the top level folder under "BLOB CONTAINERS"
        """
        blob_service_client = BlobServiceClient.from_connection_string(
            connection_string
        )
        self.container_client = blob_service_client.get_container_client(container_name)
        self.connection_string = connection_string
        self.container_name = container_name

    def upload_png(self, file_path, blob_path):
        """Uploads a figure to blob storage. Will overwrite existing files
        :param scenario_id:str number of scenario for folder name
        :param file_path:str location of png file to be uploaded
        :param blob_path:str location to put the blob in azure
            e.g. if container_name is "scenarios", blob path might be
            something like "2030_western_overbuild/emissions.png"
            thus the final blob location would be
            "scenarios/2030_western_overbuild/emissions.png"
        """
        print(f"Uploading {file_path} to {self.container_name}/{blob_path}")

        blob = BlobClient.from_connection_string(
            conn_str=self.connection_string,
            container_name=self.container_name,
            blob_name=blob_path,
        )
        try:
            with open(file_path, "rb") as data:
                blob.upload_blob(data, overwrite=True)
                blob.set_http_headers(
                    content_settings=ContentSettings(content_type="image/png")
                )
        except Exception as e:
            print(e)

        print("Upload complete")

    def upload_gzip(self, file_path, blob_path):
        """Uploads a json gzip file to blob storage. Will overwrite existing files
        :param scenario_id:str number of scenario for folder name
        :param file_path:str location of png file to be uploaded
        :param blob_path:str location to put the blob in azure
            e.g. if container_name is "scenarios", blob path might be
            something like "2030_western_overbuild/emissions.png"
            thus the final blob location would be
            "scenarios/2030_western_overbuild/emissions.png"
        """
        print(f"Uploading {file_path} to {self.container_name}/{blob_path}")

        blob = BlobClient.from_connection_string(
            conn_str=self.connection_string,
            container_name=self.container_name,
            blob_name=blob_path,
        )
        try:
            with open(file_path, "rb") as data:
                blob.upload_blob(data, overwrite=True)
                blob.set_http_headers(
                    content_settings=ContentSettings(
                        content_type="application/x-gzip", content_encoding="gzip"
                    )
                )
        except Exception as e:
            print(e)

        print("Upload complete")

    def upload_dict_as_json_gzip(self, data, local_save_path, blob_path):
        """Takes a dictionary, turns it into JSON, gzips it,
           saves the file locally, and uploads it to blob storage.
           Will overwrite existing files.
        :param data:dict the dictionary to be uploaded
        :param local_save_path:str location to save the gzipped json on your
            local computer
        :param blob_path:str location to put the blob in azure
                e.g. if container_name is "scenarios", blob path might be
                something like "2030_western_overbuild/emissions.png"
                thus the final blob location would be
                "scenarios/2030_western_overbuild/emissions.png"
        TODO: optionally do this without saving a file
        """
        json_str = json.dumps(data) + "\n"
        json_bytes = json_str.encode("utf-8")

        print(f"Saving file to {local_save_path}")
        with gzip.GzipFile(local_save_path, "wb") as fout:
            fout.write(json_bytes)

        self.upload_gzip(local_save_path, blob_path)

    def upload_bokeh_figure_as_json_gzip(
        self, figure, string_identifier, local_save_path, blob_path
    ):
        """Takes a bokeh figure, turns it into JSON, gzips it,
           saves the file locally, and uploads it to blob storage.
           Will overwrite existing files.
        :param figure:bokeh.model.Model figure generated by bokeh
        :param string_identifier:str bokeh uses this to attach the figure to
            html on the front end.
        :param local_save_path:str location to save the gzipped json on your
            local computer
        :param blob_path:str location to put the blob in azure
                e.g. if container_name is "scenarios", blob path might be
                something like "2030_western_overbuild/emissions.png"
                thus the final blob location would be
                "scenarios/2030_western_overbuild/emissions.png"
        TODO: optionally do this without saving a file
        NOTE: MAKE SURE YOU ARE USING THE CORRECT VERSION OF BOKEH from requirements.txt
            If you have the wrong version this will silently fail until we try
            to load the graph on the front end and then... boom
        """

        json_str = json.dumps(json_item(figure, string_identifier)) + "\n"
        json_bytes = json_str.encode("utf-8")

        print(f"Saving file to {local_save_path}")
        with gzip.GzipFile(local_save_path, "wb") as fout:
            fout.write(json_bytes)

        self.upload_gzip(local_save_path, blob_path)

    def list_blobs_in_container(self):
        """Lists the blobs that exist in the container
        :return: enumerable list of blob info
        """
        blobs = self.container_client.list_blobs()
        blob_list = list(blobs)
        for blob in blob_list:
            print("\t" + blob.name)
        return blob_list

In [27]:
blobs = BlobUtil("","misc")

In [32]:
i=1
blobs.upload_gzip(f'util{i}.arrow.gz', f'hourly/util{i}.arrow')

Uploading util1.arrow.gz to misc/hourly/util1.arrow
Upload complete


In [ ]:
 for i in range(int(pf_usa.shape[0]/24)):
    blobs.upload_gzip(f'util{i+1}.arrow.gz', f'hourly/util{i+1}.arrow.gz')

Uploading util1.arrow.gz to misc/hourly/util1.arrow.gz
Upload complete
Uploading util2.arrow.gz to misc/hourly/util2.arrow.gz
Upload complete
Uploading util3.arrow.gz to misc/hourly/util3.arrow.gz
Upload complete
Uploading util4.arrow.gz to misc/hourly/util4.arrow.gz
Upload complete
Uploading util5.arrow.gz to misc/hourly/util5.arrow.gz
Upload complete
Uploading util6.arrow.gz to misc/hourly/util6.arrow.gz
Upload complete
Uploading util7.arrow.gz to misc/hourly/util7.arrow.gz
Upload complete
Uploading util8.arrow.gz to misc/hourly/util8.arrow.gz
Upload complete
Uploading util9.arrow.gz to misc/hourly/util9.arrow.gz
Upload complete
Uploading util10.arrow.gz to misc/hourly/util10.arrow.gz
Upload complete
Uploading util11.arrow.gz to misc/hourly/util11.arrow.gz
Upload complete
Uploading util12.arrow.gz to misc/hourly/util12.arrow.gz
Upload complete
Uploading util13.arrow.gz to misc/hourly/util13.arrow.gz
Upload complete
Uploading util14.arrow.gz to misc/hourly/util14.arrow.gz
Upload compl